### File to use the trained model on yoga poses to predict the real-time pose of a person
#### Author: Shlok Arjun Marathe
#### Date: 6th December 2024

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import torch
from torchvision import transforms, models
from PIL import Image

In [2]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    # Use the dot product to calculate the angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)


In [3]:
# Function to detect the correctness of the pose and provide feedback
def provide_feedback(detected_pose, landmarks, ideal_angles):
    feedback = []
    mp_pose = mp.solutions.pose

    # Check the angle of each joint
    for joint, ideal_angle in ideal_angles.items():
        if joint == "left_elbow":
            angle = calculate_angle(
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
            )
        elif joint == "right_elbow":
            angle = calculate_angle(
                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
            )
        elif joint == "left_knee":
            angle = calculate_angle(
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value],
            )
        elif joint == "right_knee":
            angle = calculate_angle(
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value],
            )

        # Check if the angle is within the acceptable range
        deviation = abs(angle - ideal_angle)
        if deviation > 15:  
            feedback.append(f"{joint} angle is off by {deviation:.1f} degrees.")
    
    # If no feedback is provided, the pose is correct
    if not feedback:
        feedback.append("Your pose looks great!")
    
    return feedback


In [4]:
# Declare the correct angles for each pose
ideal_pose_angles = {
    "Warrior": {
        "left_elbow": 180,
        "right_elbow": 180,
        "left_knee": 130,
        "right_knee": 180,
    },
    "Tree": {
        "left_elbow": 180,
        "right_elbow": 180,
        "left_knee": 180,
        "right_knee": 45,
    },
    "Plank": {
        "left_elbow": 90,
        "right_elbow": 90,
        "left_knee": 180,
        "right_knee": 180,
    },
    "Goddess": {
        "left_elbow": 90,
        "right_elbow": 90,
        "left_knee": 150,
        "right_knee": 150,
    },
    "Downward Dog": {
        "left_elbow": 180,
        "right_elbow": 180,
        "left_knee": 180,
        "right_knee": 180,
    },
}

In [5]:
# Load the model for yoga pose classification
model = models.resnet18(pretrained=False)  
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 5)  
model.load_state_dict(torch.load("yoga_pose_model.pth"))  
model.eval()

/Users/shlok/Coding/RevoltronX/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shlok/Coding/RevoltronX/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/var/folders/jl/sw3zc9fx1m50mbphy9kfgpq00000gn/T/ipykernel_56590/249025156.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
# Define the class names for the model
class_names = ["Downward Dog", "Goddess", "Plank", "Tree", "Warrior"]

In [7]:
# Define the preprocessing steps for the input image
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
# Load the Mediapipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1733675641.596310 28340501 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733675641.672012 28340939 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733675641.686953 28340940 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [9]:
# Start the webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Process the webcam feed
while True:
    success, frame = cap.read()
    if not success:
        break

    # Convert the BGR image to RGB
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    # Draw the landmarks and connections on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract landmarks
        landmarks = results.pose_landmarks.landmark
        height, width, _ = frame.shape
        landmarks = [(int(lm.x * width), int(lm.y * height)) for lm in landmarks]

        # Classify pose
        img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_tensor = preprocess(img_pil).unsqueeze(0)
        with torch.no_grad():
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)
            detected_pose = class_names[predicted.item()]

        # Provide feedback
        feedback = provide_feedback(detected_pose, landmarks, ideal_pose_angles[detected_pose])

        # Display pose name on the top-right corner
        cv2.putText(frame, f"Pose: {detected_pose}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display feedback on the frame
        y_offset = 100
        for line in feedback:
            cv2.putText(frame, line, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y_offset += 20

    # Display the frame
    cv2.imshow('Yoga Pose Detection with Feedback', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

2024-12-09 00:34:55.891 Python[56590:28340501] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-12-09 00:34:57.441 Python[56590:28340501] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/jl/sw3zc9fx1m50mbphy9kfgpq00000gn/T/org.python.python.savedState
W0000 00:00:1733675697.658456 28340939 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2024-12-09 00:34:57.910 Python[56590:28340501] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-09 00:34:57.910 Python[56590:28340501] +[IMKInputSession subclass]: chose IMKInputSession_Modern
